# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
%load_ext dotenv
%dotenv 

In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob
import pandas as pd

PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)
dd_px = dd.read_parquet(parquet_files).set_index("ticker")

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
dd_feat = (
    dd_px
        .groupby('ticker', group_keys=False)
        .apply(
            lambda x: x.sort_values('Date', ascending = True)
                       .assign(Close_lag_1 = x['Close'].shift(1),
                               Adj_close_lag_1 = x['Adj Close'].shift(1),
                               hi_lo_range = x['High'] - x['Low'],
                               returns = x['Close']/x['Close'].shift(1) - 1), 
            meta = pd.DataFrame(data ={'Date': 'datetime64[ns]',
                    'Open': 'f8',
                    'High': 'f8',
                    'Low': 'f8',
                    'Close': 'f8',
                    'Adj Close': 'f8',
                    'Volume': 'i8',
                    'source': 'object',
                    'Year': 'int32',
                    'Close_lag_1': 'f8',
                    'Adj_close_lag_1': 'f8',
                    'hi_lo_range': 'f8',
                    'returns': 'f8'},
                    index = pd.Index([], dtype=pd.StringDtype(), name='ticker'))
    ))

+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [5]:
# pd_feat = pd.DataFrame(dd_feat, columns = dd_feat.columns)
pd_feat = dd_feat.compute()
type(pd_feat)

pandas.core.frame.DataFrame

In [ ]:
# sz = 10
# pd_feat = (
#     pd_feat
#       .groupby('ticker', group_keys=False)
#       .apply(lambda x: x.sort_values('Date', ascending = True)
#              .assign(returns_10dma = x['returns'].rolling(window = sz).mean()))
# )
# pd_feat.head(15)

In [6]:
sz = 10
pd_feat = pd_feat.sort_values(['ticker','Date']) # sort globally by ticker then date
pd_feat['returns_10dma'] = (
    pd_feat
        .groupby('ticker', group_keys = False)['returns']
        .transform(lambda x: x.rolling(window = sz).mean())
    )
# pd_feat = pd_feat.sort_index() # optionally restore original row order
pd_feat.head(15)

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_close_lag_1,hi_lo_range,returns,returns_10dma
ticker,,,,,,,,,,,,,,
A,1999-11-18,32.546494,35.765381,28.612303,31.473534,27.068665,62546300.0,A.csv,1999,NaN,NaN,7.153078,NaN,NaN
A,1999-11-19,30.713520,30.758226,28.478184,28.880543,24.838577,15234100.0,A.csv,1999,31.473534,27.068665,2.280043,-0.082386,NaN
A,1999-11-22,29.551144,31.473534,28.657009,31.473534,27.068665,6577800.0,A.csv,1999,28.880543,24.838577,2.816525,0.089783,NaN
A,1999-11-23,30.400572,31.205294,28.612303,28.612303,24.607880,5975600.0,A.csv,1999,31.473534,27.068665,2.592991,-0.090909,NaN
A,1999-11-24,28.701717,29.998211,28.612303,29.372318,25.261524,4843200.0,A.csv,1999,28.612303,24.607880,1.385908,0.026563,NaN
A,1999-11-26,29.238197,29.685265,29.148785,29.461731,25.338428,1729400.0,A.csv,1999,29.372318,25.261524,0.536480,0.003044,NaN
A,1999-11-29,29.327610,30.355865,29.014664,30.132332,25.915169,4074700.0,A.csv,1999,29.461731,25.338428,1.341202,0.022762,NaN
A,1999-11-30,30.042919,30.713520,29.282904,30.177038,25.953619,4310000.0,A.csv,1999,30.132332,25.915169,1.430616,0.001484,NaN
A,1999-12-01,30.177038,31.071173,29.953505,30.713520,26.415012,2957300.0,A.csv,1999,30.177038,25.953619,1.117668,0.017778,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

> No, it is not necessary to convert to pandas to calculate the moving average return (although I could not figure out how to do it).
>
> Dask is more efficient when working with large datasets that may cause memory issues.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.